In [16]:
import pandas as pd
df = pd.read_csv('game_data.csv', index_col=0)

cols = ['Player Type','Player Levels','Opponent Type','Opponent Levels','Player Score','Opponent Score','Score Differential','Player Pieces Left','Opponent Pieces Left','# rounds','w1 - score per opp dot','w2 - opp dot dist from player start','w3 - score per opp dot open corners #','w4 - score per player dot','w5 - player dot dist from opp start','w6 - score per player dot open corners #','w7 - player score multiplier','w8 - opponent score multiplier','w9 - piece difficulty weight','only 5s rounds','rounds choosing only difficult pieces','# of difficult pieces included']

print(df.head)


<bound method NDFrame.head of                                                  Player Levels Opponent Type  \
Player Type                                                                    
playSmart_v2                                          [[0, 1]]    randomTurn   
randomTurn                                            [[0, 1]]  playSmart_v2   
randomTurn                                            [[0, 1]]  playSmart_v1   
playSmart_v1                                          [[0, 1]]    randomTurn   
playSmart_v1                                          [[0, 1]]  playSmart_v2   
playSmart_v2                                          [[0, 1]]  playSmart_v1   
playSmart_v1                                          [[0, 1]]  playSmart_v1   
playSmart_v1                       [[129, 1], [84, 2], [0, 3]]  playSmart_v1   
randomTurn                                            [[0, 1]]  playSmart_v1   
playSmart_v1                                          [[0, 1]]    randomTurn   
randomTurn